# AutoGluon-Tabularを用いたシンプルな機械学習モデルの開発

このノートブックでは、AutoGluon-Tabular による高精度な機械学習モデル構築の（半）自動化をご体験頂きます。
データと予測カラム名を指定するだけでもデフォルト設定でモデル構築できますが、カスタマイズしたい点が色々と出てくるかと思います。どのようなカスタマイズができるかについてもいくつかご紹介したいと思います。

## 準備
必要となるライブラリをインストールします。

In [ ]:
# 必要なライブラリをインストールします
!pip install --upgrade mxnet
!pip install autogluon

ライブラリをインポートします。

In [ ]:
from autogluon import TabularPrediction as task
from IPython.display import HTML

## データの取得
このサンプルでは、ある人の年収が50K USDを超えるかどうかを二値分類する機械学習モデルを構築します。そのためのデータをダウンロードし、学習用データを準備します。今回は最初の500サンプルだけを利用します。

In [ ]:
train_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
train_data = train_data.head(500)
train_data.head()

予測対象となる `class` 列に格納されているデータについて確認してみましょう。

In [ ]:
label_column = 'class'

print("Summary of class variable: \n", train_data[label_column].describe())

50K USD以下の方が394人となっていることが確認できます。

## デフォルト設定での学習
このデータを用いて学習を行います。デフォルトでは、精度とコスト（メモリ使用量や推論速度等）においてバランスが取られた設定となっています。もし精度を優先してモデル構築したい場合には、 `fit()` メソッドの引数で、`preset='best_quality'` を使います。この後、このオプションでの実行も行います。詳細は[こちら](https://autogluon.mxnet.io/api/autogluon.task.html#autogluon.task.TabularPrediction.fit)をご確認下さい。

### 学習
まずは、特にオプション設定をせずに学習を行います。基準となる評価指標は `accuracy` で、presetsは `medium_quality_faster_train` です。

In [ ]:
# 学習したモデルを保存するディレクトリを指定します。
default_savedir = 'ag_models_default'

predictor = task.fit(train_data=train_data, label=label_column, output_directory=default_savedir)

学習の過程においてどのような探索を行ったか、確認してみましょう。

In [ ]:
results = predictor.fit_summary()

[`Bokeh`](https://docs.bokeh.org/en/latest/index.html) がインストールされていれば、学習の結果は HTML ファイルとして、`output_directory` へ指定したディレクトリに保存されるかと思います。SageMakerのノートブックインスタンス上で、`conda_mxnet_p36` のカーネルを選択して頂いていれば、インストールされているかと思います。

In [ ]:
# Bokehがインストール済みの場合
HTML(filename='ag_models_default/SummaryOfModels.html')

### 推論
推論のためのテストデータをダウンロードし、確認してみましょう。

In [ ]:
test_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label_column]
test_data.head()

`leaderboard` メソッドを使うと学習の過程で生成されたそれぞれのモデルについて、テストデータ、バリデーションデータでの性能、時間などが表示されます。

In [ ]:
leaderboard = predictor.leaderboard(test_data)
leaderboard

テストデータからclassカラムを抜いたデータを用いて推論を行います。もし、学習済モデルを使用する場合には、`predictor = task.load(savedir)` のように保存用ディレクトリからロードすることができます。

In [ ]:
_test_data = test_data.drop(labels=[label_column], axis=1)
y_pred = predictor.predict(_test_data)

今回学習させたモデルについて、その精度を評価してみましょう。

In [ ]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

##  精度を重視したモデル構築
ここでは、presetsオプションで、`best_quality` を指定して精度を重視したモデルを構築してみます。学習されるモデルがどう変わるのか、学習や推論にかかる時間、精度がどう変わるのかといったことを比較してみて下さい。

### 学習

In [ ]:
# 学習したモデルを保存するディレクトリを指定します。
best_savedir = 'ag_models_best'

predictor = task.fit(train_data=train_data, label=label_column, output_directory=best_savedir, presets='best_quality')

In [ ]:
results = predictor.fit_summary()

In [ ]:
# Bokehがインストール済みの場合
HTML(filename='ag_models_best/SummaryOfModels.html')

### 推論
同様に、`leaderborad` メソッドや `evaluate_predictions` メソッドを使ってモデルの評価を見てみましょう。

In [ ]:
leaderboard = predictor.leaderboard(test_data)
leaderboard

今回学習させたモデルについて、その精度を評価してみましょう。先ほどのデフォルト設定の時と比べて、精度はどう変わったか見てみましょう。

In [ ]:
y_pred = predictor.predict(_test_data)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

デフォルト設定よりも学習時間が増えています。学習時間を短くするために、`time_limits`オプションを指定してみましょう。

## 時間制限の目安を指定したモデル構築

### 学習
先程と同様に presets オプションで `best_quality` を指定し、`time_limits` オプションを指定することで、学習を目安時間内で行うよう指定できます。

In [ ]:
# 学習したモデルを保存するディレクトリを指定します。
best_time_savedir = 'ag_models_best_time'

# 時間制限を20秒にして学習を実行してみます。この時間は目安です。
time_limits = 20
predictor = task.fit(train_data=train_data, label=label_column, output_directory=best_time_savedir, presets='best_quality', time_limits =time_limits )

In [ ]:
results = predictor.fit_summary()

In [ ]:
# Bokehがインストール済みの場合
HTML(filename='ag_models_best_time/SummaryOfModels.html')

### 推論

In [ ]:
leaderboard = predictor.leaderboard(test_data)
leaderboard

In [ ]:
y_pred = predictor.predict(_test_data)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

時間制限を設けなかった場合と比べて `accuracy` はどう変わったでしょうか。

## AUCを評価指標とした場合
AutoGluonでは、ハイパーパラメータやアンサンブルの重みなどをバリデーションデータでの評価指標に基づいてチューニングします。 二値分類の際、デフォルトでは `accuracy` が評価指標ですが、`log_loss` や `f1` へ変えることもできます。また、独自の指標を活用することも出来ます。今回は評価指標を `AUC` に変えてみましょう。どのような指標を選択できるか詳しくは[こちら](https://autogluon.mxnet.io/api/autogluon.task.html#autogluon.task.TabularPrediction.fit)をご確認下さい。

### 学習

In [ ]:
# 学習したモデルを保存するディレクトリを指定します。
savedir = 'ag_models_auc'

# 評価指標を指定します。
eval_metric = 'roc_auc'
predictor = task.fit(train_data=train_data, label=label_column, eval_metric=eval_metric, output_directory=savedir)

In [ ]:
results = predictor.fit_summary()

In [ ]:
# Bokehがインストール済みの場合
HTML(filename='ag_models_auc/SummaryOfModels.html')

### 推論

これまでと同様に推論を実行しモデルの評価を確認します。

In [ ]:
leaderboard = predictor.leaderboard(test_data)
leaderboard

`AUC` を表示します。ここで使っている `evaluate` メソッドは `predict` メソッドと、 `evaluate_prediction` メソッドの短縮形です。

In [ ]:
performance = predictor.evaluate(test_data)

## モデルの解釈
学習されたモデルの解釈を行います。
下記のメソッドで計算される特徴量の重要度スコアは、その特徴量がランダムにシャッフルされた場合の性能の低下度合いです。特徴量スコアが0.01であるというのは、この性能の低下が0.01であるということです。特徴量のスコアが高いほど、その特徴量がモデルの性能に重要であると考えられます。スコアがマイナスの場合、その特徴量がモデルの性能にとってマイナスである可能性があります。くわしくは<a href='https://autogluon.mxnet.io/api/autogluon.task.html#autogluon.task.tabular_prediction.TabularPredictor.feature_importance'>こちら</a>をご確認ください。

In [ ]:
predictor.feature_importance(test_data)